In [1]:
## GEOG458
## Xinbo Lu, Wenjin Yu, Jody Tran, Doyle Park, Abraham Im
## 2/15/2019
## Group Lab 3
import sys
sys.path.append('D:\\ArcMap\\software\\Desktop10.5\\bin')
sys.path.append('D:\\ArcMap\\software\\Desktop10.5\\arcpy')
sys.path.append('D:\\ArcMap\\software\\Desktop10.5\\ArcToolbox\\Scripts')


In [24]:
import arcpy
import geopandas as gpd
import matplotlib as plt
from subprocess import call

In [3]:
arcpy.env.overwriteOutput = True
arcpy.env.workspace = "C:/Users/LXB19/Documents/GEOG458/GroupLab3/shp"

In [20]:
block_file = "saep_bg10.shp"
table_file = "WashingtonFIPS.dbf"

In [25]:
def createCounty(table_file):
    cursor = arcpy.da.SearchCursor(table_file, ["FIPSCounty","CountyName"])
    
    for row in cursor:
        output_name = row[1] + ".shp"
        input_name = "C:/Users/LXB19/Documents/GEOG458/GroupLab3/shp/" + row[0] + ".shp"
        output_name = "C:/Users/LXB19/Documents/GEOG458/GroupLab3/" + row[0] + ".geojson" # To output to the output folder.

createCounty(table_file)

WindowsError: [Error 5] Access is denied

In [ ]:
cursor = arcpy.da.SearchCursor(WA_county, ['COUNTYFP10', 'POP2017'])